In [1]:
import numpy as np
import pandas as pd
from  urlparse import urljoin 
import requests 
import bs4 as bs
from urllib2 import urlopen as req 
import sqlite3 as sql
import re
from datetime import datetime
from __future__ import unicode_literals

In [2]:
conn = sql.connect('data.db')
c = conn.cursor()

c.execute("CREATE TABLE IF NOT EXISTS URLs (url text)")

c.execute("CREATE TABLE IF NOT EXISTS reviews (name text, review text)")

In [3]:
end_point = "https://www.yelp.com/search?"
search_term = "Restaurants"
city = "Hamilton"
province = "ON"
country = "Canada"
sorting = "rating"
businesses = []

#print(datetime.now())

for page_number in range(0,91, 10):

    listing_url = (end_point + "find_desc=" + search_term + "&find_loc=" + city + ",+" + province + ",+" + country
    + "&start=" + str(page_number)  )
    
    source = req(listing_url)
    soup = bs.BeautifulSoup(source ,'lxml') 
    businesses += soup.find_all('a', {'class': 'biz-name'})
    
links = [link['href'] for link in businesses]

links[:10]

#print(datetime.now())
    
    

['/adredir?ad_business_id=wwmQOdLGXkU0TevmN1fWXg&campaign_id=BY989BOWgbfbBVUFMI1BUQ&click_origin=search_results&placement=above_search&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fatami-sushi-hamilton&request_id=6edf38b8095c138b&signature=2b97f96bbb21762add39d45de264321a2b7097c591bf6038e6d9eda43aefb21b&slot=0',
 '/biz/rapscallion-rogue-eatery-hamilton?osq=Restaurants',
 '/biz/earth-to-table-bread-bar-hamilton?osq=Restaurants',
 '/biz/charred-chicken-hamilton?osq=Restaurants',
 '/biz/the-ship-hamilton?osq=Restaurants',
 '/biz/aberdeen-tavern-hamilton-2?osq=Restaurants',
 '/biz/mezcal-tacos-and-tequila-hamilton?osq=Restaurants',
 '/biz/fsh-and-chp-hamilton?osq=Restaurants',
 '/biz/berkeley-north-hamilton-2?osq=Restaurants',
 '/biz/541-eatery-and-exchange-hamilton?osq=Restaurants']

In [4]:
regex = re.compile("^/adredir")
business_links = [i for i in links if not regex.search(i)]


In [5]:
business_links[0].split('?')[0].split('/')[-1].replace("-", " ").replace("hamilton", " ")

u'rapscallion rogue eatery  '

In [6]:
c.execute("INSERT INTO URLs (url) VALUES ('{}')".format("'), ('".join(business_links)))
conn.commit()

In [7]:
res = conn.execute("SELECT * FROM URLs")
restaurants = [_[0] for _ in res.fetchall()]
restaurants[0:10]

[u'/biz/rapscallion-rogue-eatery-hamilton?osq=Restaurants',
 u'/biz/earth-to-table-bread-bar-hamilton?osq=Restaurants',
 u'/biz/charred-chicken-hamilton?osq=Restaurants',
 u'/biz/the-ship-hamilton?osq=Restaurants',
 u'/biz/aberdeen-tavern-hamilton-2?osq=Restaurants',
 u'/biz/mezcal-tacos-and-tequila-hamilton?osq=Restaurants',
 u'/biz/fsh-and-chp-hamilton?osq=Restaurants',
 u'/biz/berkeley-north-hamilton-2?osq=Restaurants',
 u'/biz/541-eatery-and-exchange-hamilton?osq=Restaurants',
 u'/biz/pokeh-hamilton?osq=Restaurants']

In [8]:
review_divs = []
for restaurant in restaurants:

    source = req('https://www.yelp.com/' + restaurant)
    soup = bs.BeautifulSoup(source ,'lxml') 
    review_divs = soup.find_all('div', {'class': 'review-content'})
    name = soup.find_all('h1', {'class': 'biz-page-title'})[0].text.strip()
    reviews = [review.find_all('p')[0].text for review in review_divs]
    
#     print('INSERT INTO reviews (name, review) VALUES {}, "{}")'.format(
#         '("' + name + '"' ,
#         (( '"),("' + name + '","').join(
#             reviews))))

    c.executemany('INSERT INTO reviews (name, review) VALUES (?,?)', zip([name]*len(reviews), reviews))
    conn.commit()


In [9]:
result = conn.execute("SELECT * FROM reviews")
rev = [_ for _ in result.fetchall()]
len(rev)
rev[0:10]

[(u'Rapscallion Rogue Eatery',
  u"Honestly, I had driven by this place about a million times on my route to work, and walked by it probably a hundred more during my forays to Augusta. It wasn't until my friend suggested it for a ladies' night that I started reading the reviews and figured we could try it - especially as they had a special that night for four plates, two glasses of wine, and a dessert for $69.After reading the reviews, I decided that the bone marrow was definitely happening. I'd never had it before, and, really, how often do you get to actually experience gooey meat jam? Reservation in hand, off we went. And seriously? Holy effing hell. Each bite was incredibly flavourful, cooked perfectly, and you can tell that they put some serious thought and effort into each dish because they are wonderfully balanced. The menu does rotate, so that is something to keep in mind in case you want to try something specific that has been reviewed, but each thing is amazing. Also, trust y